In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1590651543638_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
path = "s3://analytics-seoul-seung/amazon-review-2010/"
df = spark.read.parquet(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(customer_id='45237643', review_id='R3NLIIZTXW9QJ9', product_id='0670021652', product_title='Committed: A Skeptic Makes Peace with Marriage', star_rating=1, review_headline="how can I review, still haven 't gotten it !!!", year=2010, review_date=14704, product_category='Books')

In [4]:
df.first()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(customer_id='45237643', review_id='R3NLIIZTXW9QJ9', product_id='0670021652', product_title='Committed: A Skeptic Makes Peace with Marriage', star_rating=1, review_headline="how can I review, still haven 't gotten it !!!", year=2010, review_date=14704, product_category='Books')

In [5]:
spark.version

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2.4.3'

In [6]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4119699

In [7]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------+----------+--------------------+-----------+--------------------+----+-----------+----------------+
|customer_id|     review_id|product_id|       product_title|star_rating|     review_headline|year|review_date|product_category|
+-----------+--------------+----------+--------------------+-----------+--------------------+----+-----------+----------------+
|   45237643|R3NLIIZTXW9QJ9|0670021652|Committed: A Skep...|          1|how can I review,...|2010|      14704|           Books|
|   11422248|R3GXFZRV0OP8H8|0758238789|           Probation|          5|       Amazing Read!|2010|      14704|           Books|
|   14569994|R2HZ7XBE8JJ6HF|1906093296|Profession and Pu...|          5|Amazing Resource ...|2010|      14704|           Books|
|   20642654|R2BWP9H3WQJEKQ|0061993999|    Not Without Hope|          5|    Not without hope|2010|      14704|           Books|
|   43879637|R2Z5HE38J9BMCH|B002YX0G0A|The Unforced Erro...|          4|New frame around ...|2010|      

In [ ]:
df.printSchema()

In [9]:
df.groupBy(df.product_category).avg("star_rating").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+
|product_category|  avg(star_rating)|
+----------------+------------------+
|              PC|  3.96669414981577|
|         Kitchen| 3.974809947171756|
|            Home|3.9363019081172244|
|        Wireless|3.6646962677120842|
|           Video| 4.212915896487985|
+----------------+------------------+
only showing top 5 rows

In [20]:
df.groupBy(df.product_category).count().orderBy("count", ascending=False).show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|    product_category|  count|
+--------------------+-------+
|               Books|1161966|
|           Video_DVD| 358626|
|               Music| 241201|
|                  PC| 214677|
|            Wireless| 162742|
|             Kitchen| 139698|
|                Toys| 131184|
|         Electronics| 125012|
|                Home| 113253|
|Digital_Ebook_Pur...| 107508|
+--------------------+-------+
only showing top 10 rows

In [11]:
df.groupBy(df.product_id).avg("star_rating").orderBy("avg(star_rating)", ascending=True).show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+
|product_id|avg(star_rating)|
+----------+----------------+
|1559361190|             1.0|
|0553592750|             1.0|
|1932494405|             1.0|
|0679454438|             1.0|
|6130344767|             1.0|
+----------+----------------+
only showing top 5 rows

In [ ]:
df.groupBy(df.product_category).agg(sum("customer_id"), avg("star_rating"))
from pyspark.sql import functions as F
df.groupBy(df.product_category).agg({'customer_id':'count', 'star_rating':'avg'}


from pyspark.sql import functions as F
df.groupBy("Profession").agg(F.mean('Age'), F.count('Age')).show()
If you're able to use different columns:

df.groupBy("Profession").agg({'Age':'avg', 'Gender':'count'}).show()


In [13]:
#df.groupBy("product_category").agg(sum("star_rating")).show()
avgDF = df.groupBy("product_category").agg({"star_rating": "avg", "customer_id": "sum"}).orderBy("avg(star_rating)", ascending=False)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
avgDF = avgDF.withColumnRenamed('avg(star_rating)', 'avg', ).withColumnRenamed('sum(customer_id)', 'sum')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#df.groupBy("product_category").count().show()

In [15]:
avgDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product_category: string (nullable = true)
 |-- avg: double (nullable = true)
 |-- sum: double (nullable = true)

In [ ]:
writePath = "s3://analytics-seoul-seung/amazon-review-2010-result2/"

avgDF.write.mode('overwrite').partitionBy('product_category').parquet(writePath)

In [ ]:
checkDF = spark.read.parquet(writePath + "/product_category=PC")
checkDF.show()

#.where("""name like "%Spark%"""")
#       .filter(_.name.contains("Spark"))
#filter(_.name == "Warsaw")

In [ ]:
checkDF.printSchema()

In [ ]:
avgDF.printSchema()